In [20]:
import geemap
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]
easy_scurbland = [
    ee.Filter.gte("size", 60000),
    ee.Filter.gt("red", 0.9)
    ]
easy_plantation = [
    ee.Filter.lt("area", 20000),
    ee.Filter.gt("area", 1000)
]

state = "rajasthan"
district = "sirohi"
block = "pindwara"

#state = "karnataka"
#district = "raichur"
#block = "devadurga"

#state = "rajasthan"
#district = "bhilwara"
#block = "mandalgarh"

#state = "odisha"
#district = "koraput"
#block = "jaypur"

#state = "jharkhand" # have same error
#district = "dumka"
#block = "masalia"

#state = "odisha" # Local computer not run
#district = "anugul"
#block = "purunakot"

#state = "bihar"
#district = "jamui"
#block = "jamui"


#state = "madhya_pradesh"
#district = "chhatarpur"
#block = "rajnagar"

state = "maharashtra"
district = "chandrapur"
block = "sawali"

#state = "odisha" # Local computer not run
#district = "anugul"
#block = "anugul"

asset_string = state + "/" + district + "/" + block + "/" + district + "_" + block

original_boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_boundaries")
farm = original_boundaries.filter(ee.Filter.And(*easy_farm))
scrubland = original_boundaries.filter(ee.Filter.And(easy_scurbland))
plantation = original_boundaries.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))

lulc_improved = ee.Image('projects/ee-raman/assets/'+block + "_v4")
lulc_v4 = ee.Image("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_lulc_v4_1")
lulc_v3 = ee.Image("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_2023-07-01_2024-06-30_LULCmap_10m")
#lulc_devadurga = ee.Image("projects/ee-corestackdev/assets/apps/mws/karnataka/raichur/devadurga/raichur_devadurga_lulc_v4")
boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_boundaries_refined")

m = geemap.Map()
m.centerObject(lulc_v3)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '000000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '3bef34'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

#m.addLayer(lulc_improved.select("predicted_label"), vis_params_corrected, 'LULC_improved')
m.addLayer(lulc_v4.select("predicted_label"), vis_params_corrected, 'LULC_v4')
m.addLayer(lulc_v3.select("predicted_label"), vis_params_corrected, 'LULC_v3')
m.addLayer(farm, {}, 'farms_easy_positives')
m.addLayer(plantation, {}, 'plantation_easy_positives')
m.addLayer(scrubland, {}, 'scrubland_easy_positives')
m.addLayer(original_boundaries, {}, "all_boundaries")

#m.addLayer(boundaries_sirohi.filter(ee.Filter.eq('class', "scrubland")), {}, 'Boundaries_sirohi')

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…